In [2]:
import argparse
import os.path as osp
import glob

import numpy as np
import mmcv
import torch
import cv2
from mmedit.apis import init_model, restoration_inference
from mmedit.core import tensor2img

import sys
sys.path.append('/home/jinsuyoo/ResizeRight')
from resize_right import resize

import matplotlib.pyplot as plt
%matplotlib inline

config = '../configs/restorers/glean/glean_ffhq_16x.py'
checkpoint = '../pretrained_model/glean_ffhq_16x_20210527-61a3afad.pth'
video_path = '/data/jinsuyoo/datasets/MEAD/M009/video/front/happy/level_1/001.mp4'
save_path = 'result_face16_celeba'
device = 0

#img_paths = glob.glob(osp.join(dir_path, '*'))
video_reader = mmcv.VideoReader(video_path)

gt_size = 1024

data = dict(hq_raw=[], hq_path=None, hq=[], lq=[])
for frame in video_reader:
    hq_raw = np.flip(frame, axis=2)
    h, w, _ = hq_raw.shape
    hq = hq_raw[(h-gt_size)//2:(h-gt_size)//2 + gt_size, (w-gt_size)//2:(w-gt_size)//2 + gt_size, :]
    lq = resize(hq, scale_factors=1/16)
    data['hq_raw'].append(hq_raw)
    data['hq'].append(hq)
    data['lq'].append(lq)
    #print(hq_raw.shape, hq.shape, lq.shape)

#sample_hq = data['hq'][0]

#plt.figure()
#plt.imshow(sample_hq)

#gt_size = 1024
#h, w, _ = sample_hq.shape
#sample_hq_crop = sample_hq[(h-gt_size)//2:(h-gt_size)//2 + gt_size, (w-gt_size)//2:(w-gt_size)//2 + gt_size, :]

#plt.figure()
#plt.imshow(sample_hq_crop)

#print(f'raw hq size: {sample_hq.shape}, cropped hq size: {sample_hq_crop.shape}')

input('pause here')

model = init_model(
    config, checkpoint, device=torch.device('cuda', device))

##################

'''
for i, img_path in enumerate(img_paths):
    output = restoration_inference(model, img_path)
    output = tensor2img(output)
    
    filename, extension = osp.splitext(osp.basename(img_path))
    mmcv.imwrite(output, osp.join(save_path, f'{filename}_glean{extension}'))
'''
result = []
img_lq = data['lq']
for lq in img_lq:
    lq = 
    lq = lq.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(lq=lq, test_mode=True)
    output = output['output'].cpu()
    result.append(output)
    print(output.shape)
result = torch.stack(result, dim=1)
print(result.shape)

h, w = result.shape[-2:]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(save_path, fourcc, 25, (w, h))
for i in range(0, result.size(1)):
    img = tensor2img(result[:, i, :, :, :])
    video_writer.write(img.astype(np.uint8))
cv2.destroyAllWindows()
video_writer.release()
print('done')

load checkpoint from http path: http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


2022-03-23 16:45:22,571 - mmedit - INFO - Load pretrained model from http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


load checkpoint from http path: http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


2022-03-23 16:45:24,684 - mmedit - INFO - Load pretrained model from http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth
2022-03-23 16:45:26,580 - mmedit - INFO - load checkpoint from torchvision path: torchvision://vgg16


load checkpoint from local path: ../pretrained_model/glean_ffhq_16x_20210527-61a3afad.pth


2022-03-23 16:45:34,060 - mmgen - INFO - Switch to evaluation style mode: single


AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'